# A Review of Robinson's Decomposition and the R-Learner

The starting point for the [R-Learner](https://arxiv.org/abs/1712.04912)
is *Robinson's decomposition* of the treatment-specific conditional mean
outcome , $\mu(w,x) = E[Y_i \mid W_i=w, X=x]$.

$$
\begin{aligned} 
\mu(W_i, X_i) &= \beta(X_i) + \{W_i-\pi(X_i)\} \tau(X_i) && \text{ where } \\
\beta(X_i)   &= E[Y_i \mid X_i], && \\
\pi(X_i) &= P(W_i=1 \mid X_i), && \text{ and } \\ 
\tau(X_i) &= E[ Y_i \mid W_i=1, X_i] - E[Y_i \mid W_i=0, X_i]. &&
\end{aligned}
$$

The use of this decomposition goes back at least to Robinson's
[root-n-consistent semiparametric regression (Econometrica
1988)](https://econpapers.repec.org/article/ecmemetrp/v_3a56_3ay_3a1988_3ai_3a4_3ap_3a931-54.htm).
We derived it in class last time.

## The R-Learner Idea

Imagine, for a moment, that you know $\pi(x)$ and $\beta(x)$. Using
Robinson's decomposition, we see that this implies that if we know
$\tau(x)$, then we know $\mu(w,x)$ and vice-versa. 

- It's often the case that you know $\pi$, as if you're an experimenter randomizing treatment,
it's part of your experimental design.
- It's essentially never the case that you know $\beta(x)$, but just go with it for now. 

Because $\tau(x)=\mu(1,x)-\mu(0,x)$, estimating it has got to boil down to some
way of estimating $\mu$, and that's what the R-Learner does. But to do
it, it uses a clever model for $\mu$ that takes advantage of our
knowledge of $\pi$ and $\beta$. In particular, because we'd know $\mu$
if we knew $\tau$ (in addition to $\pi$ and $\beta$), we use a model
that expresses $\mu(w,x)$ as a function of $\tau(x)$ and known
quantities. Given a model $\mathcal{M}_\tau$ meant to contain $\tau$,
this is what our model for $\mu$ looks like.
$$ 
\mathcal{M}_\mu = \{ m_t(w,x) : t \in \mathcal{M}_\tau \} \quad \text{ where } \quad  m_t(w,x) = \beta(x) + \{w-\pi(x)\}t(x). 
$$
Note that $\mu(w,x)=m_{\tau}(w,x)$, i.e., $\mu$ is the function we get when we plug in $t=\tau$ to our formula for $m_t$. 

To estimate $\tau$, we'll fit a
curve curve to our observations $Y_i$ by least squares over this model
$\mathcal{M}_\mu$. 
$$
\hat\mu = \operatorname*{argmin}_{\substack{m \in \mathcal{M}_\mu}} \frac{1}{n}\sum_{i=1}^n \left\{ m(W_i,X_i) - Y_i \right\}^2.
$$
The resulting estimate $\hat\tau$ of $\tau$ will be the curve for which $\hat\mu=m_{\hat\tau}$. Or equivalently, plugging in our formula for $m_t$, 
$$
\hat\tau = \operatorname*{argmin}_{\substack{t \in \mathcal{M}_\tau}} \frac{1}{n}\sum_{i=1}^n \left[ \beta(X_i) + \{W_i-\pi(X_i)\}t(X_i) - Y_i \right]^2.
$$

### The Oracle Estimator

Before we move on, let's give a name to this estimator of $\tau$. It's
not an estimator we can use, as it requires knowledge (of $\pi$ and
$\beta$) that we don't have, but it's also not our estimation
target either. It's the estimator we'd use if someone, who is
conventionally called an *oracle*, told us what $\beta$ and $\pi$ were. 
It's common practice to call estimators like this *oracle
estimators*. We'll write $\hat\tau_{\star}$ for this oracle estimator
from now on.

Thinking about this oracle estimator lets us break down the 
question of how accurate the estimator $\hat\tau$ into two parts.

1. The question of how close the estimator we actually use is to the oracle estimator.
2. The question of how close the oracle estimator is to the actual treatment effect.

That amounts to thinking of our estimator $\hat\tau$'s error as a sum of two terms.

$$
\hat\tau - \tau = \left\{\hat\tau - \hat\tau_{\star}\right\} + \left\{\hat\tau_{\star} - \tau\right\}.
$$ 


People often talk about conditions under which the first term is much smaller than the second, 
so we can think of the actual and oracle estimators as being essentially the same. It turns 
out that you don't need to estimate $\beta$ or $\pi$ particularly well for this to happen, 
which people call the *quasi-oracle property* of the R-Learner. 

Working with fake data, we'll actually calculate all three terms in the decomposition above to see where
the R-Learner's error comes from and looks like. 

## Using the R-Learner

To use this in practice, we'll take a two stage approach. 

### First Stage

We'll estimate $\beta$ and $\pi$.^[If we know $\pi$, we'll just use it: $\hat\pi=\pi$.] Each of these is, itself, a least squares regression problem.
$$
\begin{aligned}
    \hat\beta &= \operatorname*{argmin}_{b \in \mathcal{M}_\beta} \frac{1}{\tilde n}\sum_{i=1}^{\tilde n} \{ b(\tilde X_i) - \tilde Y_i \}^2 && \text{ estimates } \quad \beta(x) = E[Y_i \mid X_i=x] \\
    \hat\pi &=   \operatorname*{argmin}_{p \in \mathcal{M}_\pi} \frac{1}{\tilde n}\sum_{i=1}^{\tilde n} \{ p(\tilde X_i) - \tilde W_i \}^2 && \text{ estimates } \quad \pi(x) = E[W_i \mid X_i=x] = P(W_i=1 \mid X_i)
\end{aligned}
$$

Here $\mathcal{M}_\beta$ and $\mathcal{M}_\pi$ are models meant
to contain $\beta$ and $\pi$ respectively. 

To simplify our analysis, we'll use different samples from the same distribution for each stage. We can just use roughly half of our data for each.
$$
\begin{aligned}
 &\{(\tilde W_i, \tilde X_i, \tilde Y_i) : i \in 1 \ldots \tilde n \}  && \text{ is our first stage sample} \\
 &\{( W_i, X_i, Y_i) : i \in 1 \ldots n \} && \text{ is our second stage sample}
\end{aligned}
$$


[^1]: When we've actually randomized treatment, we should know the
    conditional treatment probability $\pi$, so we'll use the 'perfect
    estimate' $\hat\pi=\pi$ in that case. In observational studies, we
    often make the hopeful assumption that treatment was selected solely
    based on the covariate $X_i$, which justifies analyzing the data as
    if it came from an experiment but we do not to know $\pi$ so we have
    to estimate it.

### Second Stage

We'll plug our estimates $\hat\beta$ and $\hat\pi$ into our decomposition of $\mu$ to get a special model,
$$
\mathcal{M_\mu} = \{ m_t(w,x) : t \in \mathcal{M}_{\tau} \} \quad \text{ where } \quad m_t(w,x) = \hat \beta(x) + \{w - \hat \pi(x)\}t(x).
$$ 
Then we'll estimate $\hat \mu$, or equivalently $\hat \tau$, by least squares as usual. 

$$ \hat \tau = \operatorname*{argmin}_{t \in \mathcal{\mathcal{M}_\tau}}\frac{1}{n}\sum_{i=1}^n\left[\hat\beta(X_i) + \{W_i-\hat\pi(X_i)\}t(X_i) - Y_i \right]^2. $$

We can use whatever model we want for $\mathcal{M}_\tau$. 

  - Today, we'll focus on a simple version, in which we model the treatment effect $\tau(x)$ to be constant: $\mathcal{M}_\tau = \{ t(x) = c : c \in \mathbb{R}\}$. 
  - Next time, we'll do a fancier version, in which we use the bounded variation model $\mathcal{M}_\tau = \{ t(x) : \rho_{TV}(t) \le B \}$.

## Code

Before we get into the lab, let's set up some code we'll be using. In the three blocks below, we'll do the following.

1. We'll import CVXR and the `tidyverse` libraries and do a little configuration. 
2. We'll copy some convenient little functions we've used in previous labs: the functions `invert.unique` and `prediction.function`.^[We'll modify `prediction.function` so that if you do give it a function, it just returns it. Otherwise, it assumes you're giving it a model object and returns a function thatuses `predict` to get predictions.This makes it easier to plug in $\pi$ for $\hat\pi$ if we happen to know it, or plug in both $\pi$ and $\beta$ when we're talking about the oracle estimator. ]
3. We'll copy our monotone regression and bounded variation regression code from previous labs, too. 
  - We'll use the fast version of monotone regression from the convergence rates lab. 
  - We'll use `bvreg` and `predict.bvreg` from the bounded variation lab. And also our code for automatically selecting the variation budget, `select.budget` and `bvselected`.  



In [1]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(CVXR)
})

# OSQP claims some feasible problems aren't, so we'll tell CVXR not to use it
CVXR::add_to_solver_blacklist('OSQP')  

# And we'll style our plots   
lab.theme = theme(plot.background = element_rect(fill = "transparent", colour = NA),
		    					panel.background = element_rect(fill = "transparent", colour = NA),
                  legend.background = element_rect(fill="transparent", colour = NA),
                  legend.box.background = element_rect(fill="transparent", colour = NA),
                  legend.key = element_rect(fill="transparent", colour = NA),
				panel.grid.major=element_line(color=rgb(0,0,0,.05,   maxColorValue=1)),
	      panel.grid.minor=element_line(color=rgb(0,0,0,.02,   maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))
theme_set(lab.theme)


In [2]:
invert.unique = function(x) { 
  o = order(x)
  dup = duplicated(x[o])
  inverse = rep(NA, length(x))
  inverse[o] = cumsum(!dup)
  list(elements=o[!dup], inverse=inverse)
}

prediction.function = function(model) { 
  if(class(model)=='function') { model }
  else{ function(x) { as.numeric(predict(model, newdata=data.frame(X=x))) } }
}

In [3]:
monotonereg = function(X,Y, decreasing=FALSE) {
  input = list(X=X,Y=Y)

  isoreg.model = if(decreasing) { isoreg(X,-Y) } else { isoreg(X,Y) }
  mu.hat = array(dim=length(X))
  mu.hat[isoreg.model$ord] = isoreg.model$yf * if(decreasing) { -1 } else { 1 }
  
  unique.X = invert.unique(X) 
  model = list(X=X[unique.X$elements], mu.hat=mu.hat[unique.X$elements], input=input)
  attr(model, "class") = "monotonereg"
  model
}

bvreg = function(X, Y, B=1) {
  # Step 0.
  # We check that the inputs satisfy our assumptions.
  stopifnot(length(X) == length(Y))
  input = list(X=X, Y=Y)
  n = length(X)
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)

  # Step 1.
  # We tell CVXR we're thinking about a vector of unknowns m in R^p.
  m = Variable(length(unique.X$elements))
  # and permute and duplicate these into a vector mX with n elements in correspondence with (X_1,Y_1)...(X_n,Y_n)
  mX = m[unique.X$inverse]

  # Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - mX)^2 / n)

  # Step 3.
  # We specify our constraints.
  constraints = list( sum(abs(diff(m))) <= B )

  # Step 4.
  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector mu.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  mu.hat = solved$getValue(m)

  # Step 5: a little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and mu.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.bvreg 
  # 3. we return the list
  model = list(X = X[unique.X$elements], mu.hat = mu.hat, B=B, input = input)
  attr(model, "class") = "bvreg"
  model
}

# make predictions based on piecewise-constant interpolation
# we use the curve that jumps at each observation and is otherwise constant
# that is, if X[1] < X[2] < ..., 
#   mu.hat(x) for x between X[k] and X[k+1] is mu.hat(X[k])   [case 1]
#             for x > X[k]  is mu.hat(X[k])                   [case 2]
#             for x < X[1]  is mu.hat(X[1])                   [case 3]
predict.piecewise.constant = function(model, newdata=data.frame(X=model$input$X)) {
  y = model$mu.hat; X=model$X; x=newdata$X
  # for each new data point x in newdata$X, 
  # find the closest observed X[k] left of x
  # i.e., the largest k for which X[k] <= x 
  # this covers cases 1 and 2
  # i will be a vector of these numbers k, with one for each x in newdata$X
  i = findInterval(x, X) 
  # if there is no X[k] < x, findInterval tells us k=0
  # to cover case 3, we want X[k] for k=1 when this happens.
  i[i==0] = 1
  # report the values of mu.hat(X[k]), one for each x
  y[i]
}
predict.bvreg = predict.piecewise.constant
predict.monotonereg = predict.piecewise.constant

select.budget = function(X,Y,Bs) {
  set.seed(1)
  train = sample(1:length(X)) <= length(X)/2 
  Bs = c(.1,.25,.5,1,2,4,10)
  test.errors = Bs |> map(\(B) {
    muhat = bvreg(X[train], Y[train], B=B) |> prediction.function()
    mean((Y[!train] - muhat(X[!train]))^2)
  })
  Bs[which.min(test.errors)]
}

bvselected = function(Bs) {
  function(X,Y) { 
    B = select.budget(X,Y,Bs)
    bvreg(X,Y,B=B)
  }
}

## Data and Notation

Throughout, we'll be thinking about the National Supported Work (NSW)
program we discussed in the last lecture. To get a sense of how things
are working, we'll start with the fake data we were using in class. But 
once we've looked into the estimator's behavior, we'll apply it to the real stuff, too. 

We'll use the following notation.

1.  We'll use $w$ to denote the value of a *binary-valued treatment and*
    $W_i$ for the specific value received by participant $i$. Typically
    in controlled experiments, we'll let $W_i = 1$ if participant $i$
    receives the real treatment and $W_i=0$ if they receive the control
    treatment.
2.  We'll use $x$ to denote the value of a covariate. $X_i$ will be the
    specific value associated with participant $i$. In the NSW data
    we'll be working with, $X_i$ will be income in 1974, a year that precedes treatment.
3.  We'll use $y$ to denote the value of an outcome. $Y_i$ will be the
    specific value associated with participant $i$. In our NSW data,
    $Y_i$ will be income in 1978, a year that follows treatment.

Here's code for generating fake data as in lecture.

In [4]:
pi.equal   = function(x) { rep(.5, length(x)) }
pi.unequal = function(x) { .2 + .6*(1-x) }

pi = pi.equal 
mu = function(w,x) { w*(1+sqrt(x))/2  + (1-w)*(1+x)/2 }

fake.nsw.data = function(n=200) {
      X = runif(n)
      epsilon = runif(n,min=-1,max=1)*.2
      W = rbinom(n,1, pi(X))
      Y = mu(W,X) + epsilon
      data.frame(X=X,W=W,Y=Y)
}

beta = function(x) { pi(x)*mu(1,x) + (1-pi(x))*mu(0,x) }
tau = function(x) { mu(1,x) - mu(0,x) }

Here's what it looks like. 
  - I've shown the treated (W=1) group in green and the control (W=0) group in red. 
  - The group-specific signals $\mu(1,x)$ and $\mu(0,x)$ are shown as green and red lines. 
  - The ignoring-groups signal $\beta(x)$ is shown as a black line.  
  - The treatment probability $\pi(x)$ of treatment at each $x$ is indicated by the color of the background.
    - The fraction of the background that's green at a given $x$ is proportional to $\pi(x)$.
    - The fraction of the background that's red at a given $x$ is proportional to $1-\pi(x)$.
  - The treatment effect $\tau(x)$ is the difference in height between the green and red lines.

In [ ]:
group.colors = c(rgb(248,118,109, maxColorValue=255), 
                 rgb(0,191,196,   maxColorValue=255))  

plot.twogroups = function(data, mu=NULL, pi=NULL, ylim=c(0,1.25), linewidth=.5, line.alpha=.5) {
      x = seq(0,1,by=.001)
      grid = expand.grid(X=x, W=c(0,1))
      plt = ggplot() + geom_point(aes(x=X, y=Y, color=factor(W)), alpha=.3, data=data) 
      if(!is.null(mu)) { 
        plt = plt + geom_line(aes(x=X, y=mu(W,X), color=factor(W)), alpha=.2, linewidth=linewidth, data=grid) 
      }
      if(!is.null(pi)) {
        plt = plt + geom_ribbon(aes(x=x, ymax=min(ylim)+diff(ylim)*pi(x), ymin=0),   fill=group.colors[2], alpha=.1) + 
                    geom_ribbon(aes(x=x, ymin=min(ylim)+diff(ylim)*pi(x), ymax=Inf), fill=group.colors[1], alpha=.1)
      }
      if(!is.null(mu) && !is.null(pi)) {
        beta = function(x) { pi(x)*mu(1,x) + (1-pi(x))*mu(0,x) }
        plt = plt + geom_line(aes(x=x, y=beta(x)), color='black', alpha=line.alpha/2, linewidth=linewidth) 
      }
      plt + coord_cartesian(ylim=ylim)
}

plot.twogroups(fake.nsw.data(), mu=mu, pi=pi)

Here's the real thing. I've calculated a version of it with $X$ and $Y$ scaled to be between 0 and 1. That's what we'll work with. We'll scale it back up when we plot so we can see things in terms of actual dollars.

In [6]:
nsw = read.csv('https://davidahirshberg.bitbucket.io/teaching/regression/fall2021/data/nsw.csv')
nsw.experimental = nsw[nsw$dataset=='nsw', ]
real.nsw.data =  data.frame(W=as.numeric(0+nsw.experimental$treated), 
                            X=as.numeric(nsw.experimental$income74), 
                            Y=as.numeric(nsw.experimental$income78))
scaled.nsw.data = data.frame(W=real.nsw.data$W, 
                             X=real.nsw.data$X/max(real.nsw.data$X), 
                             Y=real.nsw.data$Y/max(real.nsw.data$X))

nsw_scales = list(
  scale_x_continuous(labels=function(x) { round(x*max(real.nsw.data$X), -2) }),
  scale_y_continuous(labels=function(x) { round(x*max(real.nsw.data$Y), -2) }))

Here's what it looks like. It's not as easy to make sense of visually. 
  - For a lot of people ($i$), one or both of $X_i$ (income in 1974) and $Y_i$ (income in 1978) is exactly zero. 
  - Those people had no income in those years. 

Because we don't know $\pi$, $\mu$, or $\beta$, I'll plot estimates $\hat\pi$ and $\hat\mu$ based on monotone regression and the implied estimate of $\beta$, $\hat\beta(x)=\hat\mu(0,x)\pi(x) + \hat\mu(1,x)\{1-\pi(x)\}$.

In [ ]:
pi.hat = prediction.function(monotonereg(scaled.nsw.data$X, scaled.nsw.data$W)) 
mu.hat0 = prediction.function(monotonereg(scaled.nsw.data$X[scaled.nsw.data$W==0], scaled.nsw.data$Y[scaled.nsw.data$W==0])) 
mu.hat1 = prediction.function(monotonereg(scaled.nsw.data$X[scaled.nsw.data$W==1], scaled.nsw.data$Y[scaled.nsw.data$W==1])) 
mu.hat = function(w,x) { (1-w)*mu.hat0(x) + w*mu.hat1(x) }
beta.hat = function(x) { pi.hat(x)*mu.hat(1,x) + (1-pi.hat(x))*mu.hat(0,x) }

plot.twogroups(scaled.nsw.data, mu=mu.hat, pi=pi.hat)


# The Parametric R-Learner

One of the nice things about using parametric models is that it's often
pretty straightforward to get explicit characterizations of our
estimator. This one is no exception. The least squares estimator
$\hat \tau$ in the model with constant treatment effect satisfies the
zero-derivative condition 
$$ 
\begin{aligned}
0 &= \frac{d}{dt}\bigg|_{t=\hat\tau} \frac{1}{2n}\sum_{i=1}^{n} \left[ \hat \beta(X_i) + \{ W_i - \hat \pi(X_i) \}t - Y_i \right]^2 \\
  &= \frac{1}{n}\sum_{i=1}^n \left[ \hat \beta(X_i) + \{ W_i - \hat \pi(X_i) \}\hat\tau - Y_i \right]\{W_i - \hat\pi(X_i)\} \\
  &= \frac{1}{n} \sum_{i=1}^n \{\hat \beta(X_i) - Y_i\}\{ W_i - \hat \pi(X_i) \} + \frac{1}{n}\sum_{i=1}^n \{W_i - \hat \pi(X_i)\}^2 \hat\tau. 
\end{aligned}
$$

By a little subtraction and division, we can derive an explicit formula
for our estimate $\hat\tau$.

$$ 
\begin{aligned} 
\hat \tau &= \frac{\frac{1}{n} \sum_{i=1}^n \{ Y_i - \hat \beta(X_i) \} \{ W_i - \hat \pi(X_i) \}}{\frac{1}{n}\sum_{i=1}^n \{W_i - \hat \pi(X_i)\}^2}. \\
\end{aligned}
$$

## The Behavior of the Oracle Estimator

To make sense of what this does, let's start with the oracle estimator. Let's plug in $\hat\beta=\beta$ and $\hat\pi=\pi$. 
And think about what $Y_i$ is in terms of all this stuff. 
$$
\begin{aligned}
Y_i &= \mu(W_i,X_i) + \varepsilon_i \quad 
&& \text{ where } \quad E[\varepsilon_i \mid W_i, X_i] = 0 \\
&=\beta(X_i) + \{W_i-\pi(X_i)\}\tau(X_i) + \varepsilon_i \\
\end{aligned}
$$
Plugging these into the formula for $\hat\tau$ we derived above, we get a nice formula for the oracle estimator $\hat\tau_{\star}$.

$$
\begin{aligned}
\hat\tau_{\star}
&= \frac{\frac{1}{n} \sum_{i=1}^n \left[ \beta(X_i) + \{W_i-\pi(X_i)\}\tau(X_i) + \varepsilon_i - \beta(X_i) \right] \{ W_i - \pi(X_i) \}}{\frac{1}{n}\sum_{i=1}^n \{W_i - \pi(X_i)\}^2} \\
&= \frac{\frac{1}{n} \sum_{i=1}^n \{W_i-\pi(X_i)\}^2 \ \tau(X_i) }{\frac{1}{n}\sum_{i=1}^n \{W_i - \pi(X_i)\}^2} 
+  \frac{\frac{1}{n} \sum_{i=1}^n \{W_i-\pi(X_i)\} \varepsilon_i }{\frac{1}{n}\sum_{i=1}^n \{W_i - \pi(X_i)\}^2} 
\end{aligned}
$$

1. Our first term is a weighted average of the conditional average treatment effect $\tau(X_i)$ with weights $\alpha_i = \{W_i-\pi(X_i)\}^2/\sum_{i=1}^n \{W_i - \pi(X_i)\}^2$. 
2. Our second term is a weighted average of the noise $\varepsilon_i$. 

::: {.callout-exercise}
Suppose our treatment effect were actually constant, so $\tau(X_i)=\tau$ for all $i$. 
What would the mean (expected value) of the oracle estimator $\hat\tau_{\star}$ be?

I'll give a proof in the solution, but no need for that here.
:::

It's pretty straightforward to calculate the standard deviation of the oracle estimator given that constancy assumption, too. I'll spare you the details of the calculations, but it's not a bad exercise to do it yourself if you want practice working with expected values. Assuming that $(W_1,X_1,Y_1),...,(W_n,X_n,Y_n)$ are independent and identically distributed, we have
$$
\begin{aligned}
Var[\hat\tau_{\star}] 
&= \frac{1}{n} \times E\left[ \{W_i - \pi(X_i)\}^2 \ \sigma^2(W_i,X_i) \right] 
&& \text{ for } \ \sigma^2(w,x)=Var[Y_i \mid W_i=w,X_i=x] \\
&\approx \frac{\sigma^2}{n} \times E \ Var[W_i \mid X_i] 
&& \text{ if } \sigma^2(w,x) \approx \sigma \ \text{ for all } \ w,x 
\end{aligned}
$$
If $\tau(X_i)$ *isn't* constant, then its variance is at least this large.^[This isn't hard to show using the law of total variance conditioning on $(W_1,X_1) \ldots (W_n,X_n)$. This formula is the expected value of the conditional variance. The total variance is the sum of this and the variance of the conditional mean, which is zero when $\tau(x)$ is constant, but otherwise positive.] One important conclusion we can draw is that the standard deviation of this estimator is on the order of $1/\sqrt{n}$, like a sample average's variance. 

Let's take a look at this oracle estimator's sampling distribution. We'll generate 10,000 replications of the data at sample size $n=50$ and $n=200$ and plot histograms of the resulting estimates. We can see that the distributions are both centered around the true value of $\tau$ and that the width of the distribution basically halves when we quadruple the sample size. That's what happens when you have an unbiased estimator with standard deviation proportional to $1/\sqrt{n}$.

In [ ]:
oracle.estimates = function(n, reps=10000) { 
  pi = function(x) { .5 }
  beta = function(x) { x }
  tau  = function(x) { 1 }
  1:reps |> map_vec(\(ii) {
    X = runif(n)
    W = rbinom(n,1, pi(X))
    epsilon = rnorm(n,0,.2)
    Y = beta(X) + (W - pi(X))*tau(X) + epsilon
  
    tau.hat = mean( (Y - beta(X)) * (W - pi(X)) ) / mean( (W - pi(X))^2 )
    tau.hat
  })
}

estimates.50 = oracle.estimates(50)
estimates.200 = oracle.estimates(200)

ggplot() + geom_histogram(aes(x=estimates.50, y=after_stat(density)), bins=100, 
                          alpha=.5, fill='blue', color='black', linewidth=.05) +
           geom_histogram(aes(x=estimates.200, y=after_stat(density)), bins=100, 
                          alpha=.5, fill='magenta', color='black', linewidth=.05)

To emphasize that our oracle estimator's standard deviation is some constant times $1/\sqrt{n}$, which'll be important later, let's calculate the standard deviations our our 10,000 draws at both sample sizes and multiply by $\sqrt{n}$. We'll get basically the same number.

In [ ]:
sd(estimates.50) * sqrt(50)
sd(estimates.200) * sqrt(200)

### What Happens When the Effect is Not Constant?

Generally, the mean of the oracle estimator is roughly this *treatment-variance-weighted* average of the conditional average treatment effect.^[This is what we get when we take the expectation of the numerator and denominator in our decomposition of $\hat\tau_{\star}$, then divide. That's not the same as dividing and taking the expectation, but when the the numerator and denominator are close to their means (which the law of large numbers tells us they should be), it's a good approximation. You can use Taylor expansion to get a better approximation or bound the approximation error. [This homework exercise](https://qtm285-1.github.io/assets/homework/week5-homework.html#exr-ratio-bias) from another class I teach will talk you through it.]

$$ 
\begin{aligned}
E[\hat\tau_{\star}] &\approx \frac{E\left[ \alpha(X_i) \tau(X_i) \right]}{ E\left[ \alpha(X_i) \right]} 
&& \text{where } \alpha(X_i) = E[\{W_i-\pi(X_i)\}^2 \mid X_i] = Var[W_i \mid X_i].
\end{aligned}
$$

It's an average of the conditional average treatment effect $\tau(X_i)$
that prioritizes units which, in our sample, could plausibly have
received either treatment $W_i=1$ or $W_i=0$.
Some people call this the [Overlap-Weighted Treatment
Effect](https://academic.oup.com/aje/article/188/1/250/5090958). It is
easier to estimate than the usual unweighted average treatment effect,
in the sense that it's possible to do it with lower variance, because it
downweights comparisons of potential outcomes that we rarely observe.

If you're curious, go ahead and change the code in the cell above so $\tau(X_i)$ is not constant and see what happens.

If you want to get the unweighted average treatment effect, you can 
use *weighted least squares regression* in Stage 2. Working out the right weights isn't a bad 
exercise.

::: {.callout-exercise}
Optional. Suppose we want to use an approach like this to estimate the
unweighted average treatment effect. How can we do it?

Hint: Find a formula for $\hat\tau_{\star}$ when you use *weighted least squares regression* in Stage 2 with
some arbitrary weights $w(W_i,X_i)$ and work out what choice of $w$ cancels out the $\alpha$ s. 

:::


## Using the Parametric R-Learner


Now let's use our formula for the parametric R-Learner to estimate the
overlap-weighted average treatment effect.

$$
\hat \tau = \frac{\frac{1}{n} \sum_{i=1}^n \{\hat \beta(X_i) - Y_i\}\{ W_i - \hat \pi(X_i) \}}{\frac{1}{n}\sum_{i=1}^n \{W_i - \hat \pi(X_i)\}^2}. 
$$

We'll use monotone variation regression to estimate $\beta$ and compare
our estimate to the corresponding oracle. As in lecture, we'll use the actual value of $\pi$, like we would in an experiment.

In [10]:
oracle.nuisance.functions = list(pi.hat=pi, beta.hat=beta)
estimate.nuisance.functions = function(data, fit.model = monotonereg) {
  W = data$W
  X = data$X
  Y = data$Y
  
  pi.hat = pi 
  beta.hat = prediction.function(fit.model(X, Y))
  list(pi.hat=pi.hat, beta.hat=beta.hat)
}

constant.rlearner = function(data, nuisance.functions) {
  W = data$W
  X = data$X
  Y = data$Y
  pi.hat = nuisance.functions$pi.hat
  beta.hat = nuisance.functions$beta.hat

  tau.hat = mean( (Y - beta.hat(X)) * (W - pi.hat(X)) ) / mean( (W - pi.hat(X))^2 )
  tau.hat
}

random.split = function(data) {
  stage1 = sample(1:nrow(data)) < nrow(data)/2
  stage2 = !stage1  
  list(one=data[stage1,], two=data[stage2,])
}

In [ ]:
set.seed(0)
data = fake.nsw.data(n=800)
stage = random.split(data) 

estimated.nuisance.functions = estimate.nuisance.functions(stage$one)
tau.hat.oracle = constant.rlearner(stage$two, 
                                   oracle.nuisance.functions)
tau.hat        = constant.rlearner(stage$two, 
                                   estimated.nuisance.functions)

tau.hat
tau.hat.oracle

These numbers are pretty similar. 

Let's do a visual comparison of the two estimators. We'll plot the
corresponding curves $\hat\mu(w,x) = \hat\beta(x) + \{w-\hat\pi(x)\}\hat\tau$ and
$\hat\mu_{\star}(w,x) = \beta(x) + \{w-\pi(x)\}\tau_{\star}$
on top of the data.


In [12]:
estimated.nuisance.functions = estimate.nuisance.functions(stage$one) 
beta.hat = estimated.nuisance.functions$beta.hat
pi.hat = estimated.nuisance.functions$pi.hat
mu.hat = function(w,x) { beta.hat(x) + (w-pi.hat(x))*tau.hat }
mu.hat.oracle = function(w,x) { beta(x) + (w-pi(x))*tau.hat.oracle }

In [ ]:
grid = expand.grid(w=c(0,1), x=seq(0,1,by=.001))
data.plot = plot.twogroups(stage$two, mu, pi, linewidth=2, line.alpha=.6) + 
  geom_point(aes(x=X, y=Y, color=factor(W)), shape=1, alpha=.2, data=stage$one)  

data.plot + 
  geom_line(aes(x=x, y=mu.hat(w,x), color=factor(w)), data=grid, alpha=.6) + 
  geom_line(aes(x=x, y=beta.hat(x)), data=grid, alpha=.6)

data.plot + 
  geom_line(aes(x=x, y=mu.hat.oracle(w,x), color=factor(w)), data=grid, alpha=.6) +
  geom_line(aes(x=x, y=beta(x)), data=grid, alpha=.6)


## Comparing to the Oracle

### The Rate of Convergence
Now let's get a sense of how close our estimate is to the oracle. We'll
plot the absolute difference between the two, $\lvert \hat\tau - \tau_\star \rvert$, 
as a function of sample size. 

Because this is random, we'll do it 40 times. 

  - Each time, we'll generate a new dataset of size $n=3200$ and estimate $\hat\tau$ and $\tau_\star$
    at sample sizes 100,200,...,3200. We'll see these 40 error curves as thin 'spaghetti'. 
  - We'll also plot the average of these 40 error curves as a thicker black line. 

In [14]:
reps = 40
ns = c(100,200,400,800,1600,3200)
set.seed(0)
estimates =1:reps |> map_dfr(.id='rep', \(rr) {
  all.data = fake.nsw.data(n=max(ns))
  ns |> map_dfr(\(n) {
    data = all.data[1:n,]
    stage = random.split(data) 
    
    tau.hat.oracle = constant.rlearner(stage$two, 
                                       oracle.nuisance.functions)
    tau.hat = constant.rlearner(stage$two, 
                                estimate.nuisance.functions(stage$one))
    data.frame(estimator = tau.hat, oracle = tau.hat.oracle, n=n)
  })
})

In [ ]:
convergence.plot = 
  ggplot(estimates) + 
    geom_line(aes(x=n,    y=abs(estimator-oracle), color=rep, group=rep), linewidth=.5, alpha=.5) + 
    stat_summary(aes(x=n, y=abs(estimator-oracle)), fun.data=mean_se, geom='pointrange', linewidth=.5) + 
    stat_summary(aes(x=n, y=abs(estimator-oracle)), fun=mean, geom='line') + guides(color='none')
convergence.plot

To summarize this curve, we'll estimate a rate of convergence like 
we did in our convergence rates lab. We'll use `nls` to fit a model like this.

$$
\lvert \hat\tau - \tau_\star \rvert \approx \hat\alpha n^{-\hat\rho} \quad \text{ for some } \hat\alpha \text{ and } \hat\rho.
$$

We'll take a look at the rate and, to check that it makes sense, plot what it predicts, i.e. $\hat\alpha n^{-\hat\rho}$, on top of the actual error curves. In blue.


In [ ]:
convergence.model = nls(abs(estimator-oracle) ~ a * n^(-r),
                        data=estimates,
                        start=list(a=1, r=1/2))
convergence.model 

error.predictions = data.frame(n=ns) 
error.predictions$error.prediction = predict(convergence.model, newdata=error.predictions)
convergence.plot + 
  geom_line(aes(x=n, y=error.prediction), data=error.predictions, 
            color='blue', linewidth=2, alpha=.5)

$n^{-\hat\rho}$ is our estimate of the rate of convergence of $\hat\tau$ to $\tau_\star$.
Note that we're converging to the oracle at a rate **faster than** $n^{-1/2}$, which is the rate at which the oracle estimator
converges to **its limit**. This tells us the difference between our
estimate and the oracle is asymptotically negligible in the sense that
the actual estimate converges to the oracle estimator's limit
essentially just as fast as the oracle estimator itself does. Let's take
a look. We'll use the value of the oracle estimator at a large sample
size as a proxy for its limit. In the plot below, we see...

-   dashed lines showing the estimator at multiple sample sizes
-   solid lines showing the oracle estimator at the same sample sizes
-   a green horizontal line showing the oracle estimator's limit

We do show this, using lines of different colors, for a few different replications of the data.

In [ ]:
set.seed(0)
data = fake.nsw.data(n=100000)
tau.hat.oracle.limit  = constant.rlearner(data, oracle.nuisance.functions)

estimator.sequence.plot = ggplot(estimates[estimates$rep %in% 1:3,]) + 
           geom_line(aes(x=n, y=estimator, color=rep, group=rep), linetype='dashed') +  
           geom_point(aes(x=n, y=estimator, color=rep, group=rep)) +  
           geom_line(aes(x=n, y=oracle, color=rep, group=rep)) +
           geom_point(aes(x=n, y=oracle, color=rep, group=rep)) +  
           geom_segment(aes(x=n, xend=n, y=estimator, yend=oracle, color=rep, group=rep), alpha=.2) + 
           geom_hline(aes(yintercept=tau.hat.oracle.limit), color='green', linewidth=2, alpha=.5) + 
           guides(color='none')
estimator.sequence.plot

### Distributional Implications

What does this equivalence mean for the *sampling distribution* of our estimator?
Let's take a look. We'll use 10,000 replications of our data at sample size n=800
to histogram the sampling distributions of the oracle estimator (blue) and our actual estimator (red).
The green line shows the oracle estimator's limit. What do you see?


In [18]:
n = 800
reps = 10000

tau.hat = rep(NA, reps)
tau.hat.oracle = rep(NA, reps)

set.seed(0)
estimates = 1:reps |> map_dfr(.id='rep', \(rr) {
  data = fake.nsw.data(n=n)
  stage = random.split(data) 
  tau.hat.oracle = constant.rlearner(stage$two, 
                                     oracle.nuisance.functions)
  tau.hat = constant.rlearner(stage$two, 
                                estimate.nuisance.functions(stage$one))
  data.frame(estimator = tau.hat, oracle = tau.hat.oracle, n=n)
})

In [ ]:
nbins = 100
histogram.comparison = ggplot(estimates) + 
           geom_histogram(aes(x=estimator, y=after_stat(density)), bins=nbins, alpha=.5, fill='red', color='black', linewidth=.05) + 
           geom_histogram(aes(x=oracle, y=after_stat(density)), bins=nbins, alpha=.5, fill='blue', color='black', linewidth=.05) + 
           geom_vline(aes(xintercept=tau.hat.oracle.limit), color='green', linewidth=2, alpha=.5)
histogram.comparison

They're pretty similar, right?  To see why, let's look at the actual *differences* between the two estimators in each sample.
Here are two visualizations. 

1. We plot two little dots for each sample, one red and one blue, showing the two estimators. With a little stick connecting them. 
2. We histogram the *differences* between the two estimators. 

I've plotted these on top of the sampling distributions above to give a sense of scale.

In [ ]:
max.y = dnorm(0,sd=sd(estimates$estimator))
ys = seq(0,max.y,length.out=nrow(estimates))

histogram.comparison + 
  geom_point(aes(x=estimator, y=ys),alpha=.5, color='red', size=.1) + 
  geom_point(aes(x=oracle, y=ys), alpha=.5, color='blue', size=.1) + 
  geom_segment(aes(x=estimator, xend=oracle, y=ys, yend=ys), alpha=.2, linewidth=.1)

histogram.comparison +  geom_histogram(aes(x=estimator-oracle, y=after_stat(density)), 
                          bins=nbins, alpha=.5, fill='purple', color='black', linewidth=.05)

## The R-Learner in Observational Studies 

In observational studies, we don't know $\pi$, so we have to estimate
it. We can do this by fitting a model for $\pi$ to our data and using
the resulting estimate $\hat\pi$ in place of $\pi$ in our estimator.
Here's a theoretical claim. It's presented as an exercise, but it's a
hard one if you're not used to thinking about these things.
I'll be happy to talk you through it during office hours, but you might
want to skip it for now.

::: {.callout-exercise}
*Optional*. Show that the difference between $\hat\tau$ and
$\hat\tau_{\star}$ is negligible if the errors $\hat \pi - \pi$
and $\hat \beta - \beta$ converge to zero at the rates $n^{-\rho_\pi}$
and $n^{-\rho_\beta}$ respectively if 

1. $\rho_\beta > 0$
2. $\rho_\pi > 1/4$
3. $\rho_\beta + \rho_\pi > 1/2$.
:::

[^2]: You may assume that this means
    $\lVert \hat \pi - \pi \rVert \le a_\pi n^{-r_\pi}$ for whatever
    norm you find convenient and similarly for $\hat\beta-\beta$.

*Tip.* To show this, I suggest again starting by expanding $\hat \tau$
around $\hat\tau_{\star}$. This isn't quite as easy as the case
in which $\pi$ is known, as unlike $\hat\beta$, $\hat\pi$ occurs in the
denominator of of $\hat\tau$ as well as in the numerator. To work around
this straightforwardly, you can think of our estimators as functions $f$
of the vectors $\beta(X)=\beta(X_1)\ldots\beta(X_n)$ and
$\pi(X)=\pi(X_1)\ldots \pi(X_n)$, and look at the first order Taylor
series expansion of $\hat\tau = f(\hat\beta(X), \hat\pi(X))$ around
$\hat\tau_{\star} = f(\beta(X), \pi(X))$ and that
expansion's remainder.

That said, we don't need to know the theory to try it out. Here's
another exercise.

::: {.callout-exercise}
*Optional*. Repeat the experiments above, but use an estimate $\hat\pi$ in
place of $\pi$ itself when calculating $\hat\tau$. You can estimate $\pi$
using monotone or bounded variation regression. If you like, change $\pi$ 
from $\text{pi.unequal}$ to $\text{pi.equal}$ in the 'Data' section, repeat,
and see what happens.
:::

### Implications {#implications}

If you think about it, this is a very cool result. It says that you don't
need particularly accurate estimates of $\pi$ and $\beta$ to get an accurate 
estimate of $\tau$. In particular, if we use monotone or bounded variation regression to estimate $\pi$ and $\beta$, 
we'll get $n^{-1/3}$ rates of convergence for both $\hat\beta$ and $\hat\pi$, and that's good enough to get almost exactly the same estimate of $\tau$ as if we knew $\pi$ and $\beta$ exactly. The practical take-away is that we don't 
need to consider fitting simpler models (e.g. lines) when estimating $\pi$ and $\beta;
these offer no improvement when they're (approximately) correct, but can mess things 
up when they're not.

Later in the semester, I'll sketch a partial generalization of this result
that applies when we use a nonparametric model for $\tau$ later in the
semester. [Nie and Wager](https://arxiv.org/abs/1712.04912) call this the *Quasi-Oracle Property* of the
R-Learner. In short, it says the rate of convergence of $\hat\tau(x)$ to
$\tau(x)$ will be the same as the rate of convergence we'd get using 
the oracle estimator $\hat\tau_\star(x)$ or equivalently by 
doing least squares on direct observations least squares on direct observations
$Y^\tau_i = \tau(X_i) + \varepsilon_i^\tau$ of $\tau$ as long as this is rate
is slower than $n^{-(\rho_\pi+\rho_\beta)}$ and $n^{-2\rho_\pi}$. For example, if
$\hat\beta$ and $\hat\pi$ converge at rates faster than $n^{-1/6}$, then
if we use an R-Learner version of bounded variation regression we'll get the same
$n^{-1/3}$ rate we'd get via least squares bounded variation regression on these
direct observations. So far we don't have any methods for estimating
$\beta$ and $\pi$ that converge this slow, but we will see this happen
later in the semester when we start working with data in which $X_i$ is
higher-dimensional.


# The Nonparametric R-Learner

Now let's implement a version that uses a nonparametric model for
$\tau(x)$ in a model that actually allows it to vary with $x$. We'll use the 
bounded variation model.

$$ \hat \tau = \operatorname*{argmin}_{\substack{t \\ \rho_{TV}(t) \le B}} \frac{1}{n}\sum_{i=1}^n\left[\hat\beta(X_i) + \{W_i-\hat\pi(X_i)\}t(X_i) - Y_i \right]^2. $$

::: {.callout-exercise}
Modify the code in the block below so it fits a bounded variation model for $\tau$ using
the R-Learner. 

To test your code, run the block after it to fit it to some simulated data 
and compare it to $\tau$ itself.
:::

Here's a template. I've handled the boilerplate stuff for you, e.g. naming and saving of
variables, but all the code does as-is is fit a model to predict Y from X. What do you need to change?

In [21]:
bvrlearner = function(W, X, Y, nuisance.functions, B=1) {
  # check that the inputs make sense
  stopifnot(length(W) == length(X)) 
  stopifnot(length(X) == length(Y)) 
  input = c(list(W=W, X=X, Y=Y), nuisance.functions)
  pi.hat = nuisance.functions$pi.hat
  beta.hat = nuisance.functions$beta.hat
  # and find the unique elements of X and the inverse mapping
  unique.X = invert.unique(X)
   
  # We tell CVXR we're thinking about a vector of unknowns, t, in R^p. 
  #   - t will have one element for each unique level of X.
  #   - t[i] will correspond to the ith smallest element of X. 
  # We use invert.unique to duplicate and shuffle these into a vector of unknowns in correspondence with Y_1 ... Y_n.
  t = Variable(length(unique.X$elements))
  t.WX = t[unique.X$inverse]
  
  # And we tell it how to predict Y from W and X. 
  # in terms of our placeholder t for the unknown vector tau(X1)...tau(Xn)
  # Here's where you come in. This does something, but isn't right. Change it.
  m.WX = t.WX  

  # We tell CVXR that we're interested in mean squared error and specify the constraints.
  mse = sum((Y - m.WX)^2 / n)
  constraints = list( sum(abs(diff(t))) <= B )

  # We ask CVXR to minimize mean squared error subject to our constraints.
  # And we ask for vector tau.hat that does it.
  solved = solve(Problem(Minimize(mse), constraints))
  tau.hat = solved$getValue(t)

  # A little boilerplate to make it idiomatic R.
  # 1. we record the unique levels of X and tau.hat, in correspondence and sorted in increasing order of X, in a list. We also record the input data. 
  # 2. we assign that list a class, so R knows predict should delegate to predict.bvrlearner 
  # 3. we return the list
  model = list(X = X[unique.X$elements], tau.hat = tau.hat, B=B, input = input)
  attr(model, "class") = "bvrlearner"
  model
}

In [ ]:
set.seed(0)
data = fake.nsw.data(n=200)
stage = random.split(data) 

B = 1/2
nuisance.functions = estimate.nuisance.functions(stage$one)
tau.model = bvrlearner(stage$two$W, stage$two$X, stage$two$Y, 
                       nuisance.functions, B=B)

ggplot() + geom_point(aes(x=tau.model$X, y=tau.model$tau.hat), color='red') + 
           geom_point(aes(x=tau.model$X, y=tau(tau.model$X))) 

## Making Predictions using the R-Learner

We can use piecewise-constant extension to calculate $\hat\tau(x)$ at new points $x$. 
Here's a function that does it using our old friend `predict.piecewise.constant`.
That function expects a model with a `mu.hat` field, so we'll rename `tau.hat` to `mu.hat` before we call it.

It takes an argument 'type' to specify whether you want a prediction 
of the response (i.e. an estimate of $\mu(w,x)$) or the effect (i.e. an estimate of $\tau(x)$).

In [23]:
predict.bvrlearner = function(model, newdata=data.frame(W=model$input$W, X=model$input$X), type='response') {
  model$mu.hat = model$tau.hat
  tau.hat = predict.piecewise.constant(model, newdata)
  if(type == 'effect') { 
    tau.hat 
  } else { 
    beta.hat = model$input$beta.hat
    pi.hat = model$input$pi.hat
    beta.hat(newdata$X) + (newdata$W-pi.hat(newdata$X))*tau.hat
  }
}

## Visualization

Let's calculate the real and oracle estimates and do a visual
comparison. We'll plot the curves
$\hat\mu(w,x) = \hat\beta(x) + \{w-\pi(x)\}\hat\tau(x)$ and
$\hat\mu_{\star}(w,x) = \beta(x) + \{w-\pi(x)\}\hat\tau_{\star}(x)$.

In [24]:
set.seed(0)
data = fake.nsw.data(n=800)
stage = random.split(data) 

B = 1/2
nuisance.functions = estimate.nuisance.functions(stage$one)
tau.model =       bvrlearner(stage$two$W, stage$two$X, stage$two$Y, 
                             nuisance.functions, B=B)
tau.model.oracle = bvrlearner(stage$two$W, stage$two$X, stage$two$Y, 
                              oracle.nuisance.functions, B=B)

mu.hat = function(w,x) { predict(tau.model, newdata=data.frame(W=w, X=x), type='response') }
mu.hat.oracle = function(w,x) { predict(tau.model.oracle, newdata=data.frame(W=w, X=x), type='response') }
tau.hat = function(x) { predict(tau.model, newdata=data.frame(X=x), type='effect') }
tau.hat.oracle = function(x) { predict(tau.model.oracle, newdata=data.frame(X=x), type='effect') }

Let's take a look at the two treatment effect estimators. I'll plot the
oracle as a solid line and the actual one as a dashed line. They're very
close.

In [ ]:
xx = seq(0,1,by=.01)
ggplot() + geom_line(aes(x=xx, y=tau.hat.oracle(xx)), color='blue')  +
           geom_line(aes(x=xx, y=tau.hat(xx)), color='red')  

To get a sense of what this looks like in the context of the data, we
can look at the corresponding treatment specific means $\hat\mu(w,x)$.
Here's what those looks like. We'll plot $\hat\beta(x)$ with the actual
estimates and $\beta(x)$ with the oracle ones, too.

In [ ]:
data.plot + 
  geom_line(aes(x=x, y=nuisance.functions$beta.hat(x)), data=grid, alpha=.6) + 
  geom_line(aes(x=x, y=mu.hat(w,x), color=factor(w)), data=grid, alpha=.6) 

data.plot + 
  geom_line(aes(x=x, y=beta(x)), data=grid, alpha=.6) + 
  geom_line(aes(x=x, y=mu.hat.oracle(w,x), color=factor(w)), data=grid, alpha=.6) 

## Convergence to the Oracle

Now let's get a sense of how close our estimate is to the oracle. We'll
plot the root-mean squared distance
$\lVert \hat \tau - \hat \tau_{\star} \rVert_{L_2(P)}$ as a
function of sample size. And, for reference, the same for the difference
$\lVert \hat\tau_{\star} - \lim_{n \to \infty} \hat\tau_{\star}\rVert_{L_2(P)}$
between the oracle and its limit. Or, due to computational restrictions,
a 'limit' that's actually just the value of the oracle fit to a
large-ish sample independent of the one we're using elsewhere.

In [27]:
set.seed(2)
B = 1/2 

ns = c(200,400,800,1600,3200)
set.seed(0)
all.data = fake.nsw.data(n=max(ns))
xx = seq(0,1,by=.01)

tau.model.oracle.limit = bvrlearner(all.data$W, all.data$X, all.data$Y, oracle.nuisance.functions, B=B)
tau.hat.oracle.limit = function(x) { predict(tau.model.oracle.limit, newdata=data.frame(X=x), type='effect') }

rms.difference = rep(NA, length(ns))
rms.oracle.error = rep(NA, length(ns))
for(ii in 1:length(ns)) {
  n = ns[ii] 
  data = all.data[1:n,]
  stage = random.split(data) 

  estimated.nuisance.functions = estimate.nuisance.functions(stage$one)
  tau.model =        bvrlearner(stage$two$W, stage$two$X, stage$two$Y, 
                                estimated.nuisance.functions, B=B)
  tau.model.oracle = bvrlearner(stage$two$W, stage$two$X, stage$two$Y, 
                                oracle.nuisance.functions, B=B)

  tau.hat = function(x) { predict(tau.model, newdata=data.frame(X=x), type='effect') }
  tau.hat.oracle = function(x) { predict(tau.model.oracle, newdata=data.frame(X=x), type='effect') }

  rms.difference[ii] = sqrt(mean((tau.hat(xx) - tau.hat.oracle(xx))^2))
  rms.oracle.error[ii] = sqrt(mean((tau.hat.oracle(xx) - tau.hat.oracle.limit(xx))^2))
}


Here's a comparison of the RMS actual-oracle difference (dashed) and the
RMS oracle-'limit' difference (solid). What see is what the theory predicts. 
The difference between $\hat\tau$ and $\hat\tau_{\star}$ is much smaller than the difference between $\hat\tau_{\star}$ and its limit.


In [ ]:
convergence.plot.tv = 
  ggplot() + geom_line(aes(x=ns, y=rms.difference), linetype='dashed') + 
             geom_point(aes(x=ns, y=rms.difference)) + 
             geom_line(aes(x=ns, y=rms.oracle.error)) +
             geom_point(aes(x=ns, y=rms.oracle.error)) + ylim(0,max(rms.oracle.error))
convergence.plot.tv


To summarize what's going on, we can approximate each by a multiple of a
power of $n$. You know the drill. 

$$
\lVert \text{ difference } \rVert \approx \alpha n^{-\rho} \quad \text{ for some } \alpha \text{ and } \rho.
$$ 


In [ ]:
difference.model.tv = nls(abs(rms.difference) ~ a * ns^(-r),
                        data=data.frame(ns=ns, rms.difference=rms.difference),
                        start=list(a=1, r=1/2))
difference.model.tv.predictions = predict(difference.model.tv, newdata=data.frame(ns=ns))

convergence.model.tv = nls(abs(rms.oracle.error) ~ a * ns^(-r),
                        data=data.frame(ns=ns, rms.oracle.error=rms.oracle.error),
                        start=list(a=1, r=1/2))
convergence.model.tv.predictions = predict(convergence.model.tv, newdata=data.frame(ns=ns))

convergence.plot.tv + geom_line(aes(x=ns, y=convergence.model.tv.predictions), color='blue') +
                      geom_line(aes(x=ns, y=difference.model.tv.predictions), color='blue', linetype='dashed')

summary(difference.model.tv)
summary(convergence.model.tv)

These approximations are pretty accurate. And one way of summarizing what they say is that,
if we increase the sample size enough that the oracle estimator gets one digit of precision 
closer to its limit, then the R-Learner estimator gets two digits of precision closer to the oracle.
This is essentially what the theory tell us to expect.


## Application

Let's use what we've got to estimate the effect of the National
Supported Work (NSW) program on 1978 income as a function of 1974
income.

Let's think about how we estimate $\pi$ and $\beta$. 
  
- This is a randomized experiment, so it should be the case that $\pi(x)$ is constant. We can try to estimate that constant using `lm`. 
- You'd think higher income in '74 predicts higher income in '78, so $\beta(x)$ should be increasing, so let's estimate $\beta$ using monotone regression. 

We can see what happens when we change these, too.

To distinguish data used in stage 1 and stage 2, I've used hollow circles for stage 1 and filled ones for stage 2.

In [ ]:
data = scaled.nsw.data
set.seed(0)
stage = random.split(data)

estimated.nuisance.functions = list( 
  pi.hat   = prediction.function( lm(W ~ 1, data=stage$one) ),
  beta.hat = prediction.function( monotonereg(stage$one$X, stage$one$Y) ))
tau.model = bvrlearner(stage$two$W, stage$two$X, stage$two$Y, 
                       estimated.nuisance.functions, B=1/2)

mu.hat = function(w,x) { predict(tau.model, newdata=data.frame(W=w, X=x), type='response') }
tau.hat = function(x) { predict(tau.model, newdata=data.frame(X=x), type='effect') }

grid = expand.grid(w=c(0,1), x=seq(0,1,by=.001))

plot.twogroups(stage$two, mu.hat, estimated.nuisance.functions$pi.hat) +
  geom_point(aes(x=X, y=Y, color=factor(W)), shape=1, alpha=.3, data=stage$one) + 
  guides(color='none') + 
  nsw_scales

xx = seq(0,1,by=.001) 
ggplot() + geom_line(aes(x=xx, y=tau.hat(xx))) + nsw_scales